# 01 - Context Managers in Python

#### Introduction

(PEP 343)

Typically, the steps involved in using a context manager are the following:

1. Create a context (a minimal amount of state needed for a block of code)
2. Execute some code that uses variables from the context
3. Automatically clean up the context when we're done

Let's take a look at a simple example:

```python
with open('test.txt', 'w') as f:
    print(f.readlines())
```

- `open()`: **Creates** the context
- `with`: **enters** the context

The context closes once the indented block completes.

Context managers data in our scope **on entry** and **on exit**. 

Therefore, it's very useful for anything that needs the **Enter/Exit** or **Start/Stop** or **Set/Reset** pattern.

Examples are:
- Open / close a file.
- Start db transaction / commit or abort transaction.
- Set decimal precision to 3 / reset back to original precision
- Change / reset (the state of a class)
- Lock / release (a thread)

#### Lecture

The functionality of a context manager can be achieved with a `try`-`finally` block, but it gets quite cumbersome.

The layout is as following:
```python
with <context object> as <obj_name>:
    <with block>
<after with block; context cleaned up>
```

A `context` object is one that implements the `context management protocol`.

Classes implement this protocol by implementing two methods:
1. `__enter__`: does the setup and optionally returns some object
2. `__exit__` tear down / cleanup

Consider this context:
```python
with CtxManager() as obj:
    # do something
# done with context
```
Let's replicate the functionality of a context manager with a simplified `try`-`finally`:
```python
mgr = CtxManager()
obj = mgr.__enter__()
try:
    # do something
finally:
    # done with context
    # mgr.__exit__()
```

**Further Breakdown**

In the following code, note that `MyClass` behaves like a regular class from which objects can be instantiated until it encounters a `with` keyword.

- **It is `with` that calls the `__enter__()` method.**
- The return of `__enter__()` is passed to the object after the `as` keyword.
- To emphasise, what gets put into `obj` is **not** the return of `MyClass` but rather the return of `__enter__()`.
- `MyClass().__exit__()` only gets called after the `with` block **or** if an exception is called - the arguments of `__exit__()` are for dealing with exceptions.

```python
class MyClass:
    def __init__(self):

    def __enter__(self):
        # return obj

    def __exit__(self, <...>):
        # clean up

my_obj = MyClass()

with MyClass() as obj:
    # do something
```

**Scopes**

A `with` block does **not** have its own scope unlike a function or list comprehension. It just shares the scope of wherever it's running in. Even the `obj` in `as obj` is within this same scope.

To illustrate with a simple example:
```python
# module.py
with open(fname) as f:
    row = next(f)

print(f, row) # this will work
```
`f` and `row` are symbols in the **global** scope even after the context closes.

Note that, in the case of `open()`, the return of `open()` is the same as the return of its `__enter__()` method. This allows both approaches:
- `f = open('test.txt', 'w')`
- `with open('test.txt', 'w') as f:`

**`__enter__` and `__exit__`**

`__enter__` is quite simple; we've explained everything that we need to about it.

`__exit__` will **always** run even if an exception occurs, but if that's the case, how do we deal with it in comparison to when we normally finish our `with` block? 

    We can silence it which lets it continue the regular teardown or we can propagate it which lets it continue as if we hadn't intercepted it. The latter will terminate the entire module program thereby preventing any code below the `with` block to execute, unlike the former which will let the code continue as normal.

    The `__exit__` method needs 3 arguments:
    - exception type
    - exception value
    - traceback object (the big output that retraces our steps when we hit an exception) 

    If no exception occurs, then all three of these arguments will return `None`.

    The `__exit__` method must return `True` or `False`. 
    - `True`: silence any raised exception; continue execution as normal
    - `False`: propagate exception

#### Coding

##### Example 1

Here's a basic example:

In [5]:
class MyContext:
    def __init__(self):
        self.obj = None

    def __enter__(self):
        print('Entering context...')
        self.obj = 'The Return Object'
        return self.obj

    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exiting context...')
        if exc_type:
            print(f"*** Error occurred, manually catching: {exc_type}, {exc_value}")
        return False

Note:
- `exc_type`, `exc_value`, `exc_tb` can be named whatever we like, but the positions matter. The first for the type, the second for the value and the third for the traceback info object.
- Think of `__exit__` as *silent* or *normal* exit. Then, `return True` will silence the exception and no traceback will appear. `return False` will trap any exceptions caught and bubble it back up.

In [8]:
# since `return False`, we expect exception to bubble back up and remaining code after `with` block to not run

with MyContext() as obj:
    print(f"Inside block; {obj = }")
    raise ValueError('some value')

print('rest of program here; I wont run')

Entering context...
Inside block; obj = 'The Return Object'
exiting context...
*** Error occurred, manually catching: <class 'ValueError'>, some value


ValueError: some value

Now let's switch the return flag to `False`:

In [10]:
class MyContext:
    def __init__(self):
        self.obj = None

    def __enter__(self):
        print('Entering context...')
        self.obj = 'The Return Object'
        return self.obj

    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exiting context...')
        if exc_type:
            print(f"*** Error occurred, manually catching: {exc_type}, {exc_value}")
        return True

In [11]:
# since `return True`, we expect exception to be suppressed and remaining code after `with` block will run

with MyContext() as obj:
    print(f"Inside block; {obj = }")
    raise ValueError('some value')

print('rest of program here; I will run')

Entering context...
Inside block; obj = 'The Return Object'
exiting context...
*** Error occurred, manually catching: <class 'ValueError'>, some value
rest of program here; I will run


**Good Practices**

Often we will trap certain common exceptions, handle them and silent exit `return True`, but for the uncommon exceptions, we'll let them bubble back up (`return False`).

##### Example 2

Let's create a context manager that creates and utilises a new resource upon entering the context:

In [21]:
class Resource:
    def __init__(self, name):
        self.name = name
        self.state = None

class ResourceManager:
    def __init__(self, name):
        self.name = name
        self.resource = None  # we only want to create the resource when we enter a context

    def __enter__(self):
        print('entering context')
        self.resource = Resource(self.name)
        self.resource.state = 'Created'
        return self.resource

    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exiting context')
        self.resource.state = 'Destroyed'
        if exc_type: # if an exception has occured
            print('Exception occurred!')
        return False

In [25]:
with ResourceManager('spam') as res:
    print(f"{res.name = }, {res.state = }")

print(f"{res.name = }, {res.state = }")

entering context
res.name = 'spam', res.state = 'Created'
exiting context
res.name = 'spam', res.state = 'Destroyed'


# 02 - Caveat with Lazy Iterators

As we know, `open()` creates a context and utilises a lazy iterator. What might often happen is the context closes before we get a chance to iterate through the iterator. For example:

In [30]:
import csv

def read_data():
    with open("nyc_parking_tickets_extract.csv", 'w') as f:
        return csv.reader(f, delimiter=',', quotechar='"')

reader = read_data()
for row in reader:
    print(row)

ValueError: I/O operation on closed file.

This occurred because `return` is present in the `with` block. As a result, `with` called `open`'s `__exit__()` which closed the file. How do we get around this?

After `return` returns, there's nothing left to execute in the `with` block so the `context` can exit.

But `yield`/`yield from` doesn't exit until we've iterated through the entire iterator and hit a `StopIteration` exception. This is how we can solve our problem.

Note: `csv.reader()` is an iterator; we can either start a `for` loop that yields on each iteration or yield directly from the iterator with `yield from`.

The two lines below are *basically* identical

```python
yield from csv.reader(f, delimiter=',', quotechar='"')`
```
```python
for row in csv.reader(f, delimiter=',', quotechar='"'):
    yield row
```

In [33]:
import csv

def read_data():
    with open("nyc_parking_tickets_extract.csv") as f:
        yield from csv.reader(f, delimiter=',', quotechar='"')

reader = read_data()

for row in reader:
    print(row)
    break

['Summons Number', 'Plate ID', 'Registration State', 'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make', 'Violation Description']


# 03 - Not Just a Context Manager

# 04 - Additional Uses

# 05 - Generators and Context Managers

# 06 - The contextmanager Decorator

# 07 - Nested Context Managers